# Mini Projekt - Baby Kyber

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [108]:
import numpy as np 

In [109]:
# skopiuj pierścień ilorazowy wielomianów z pierwszych zajęć

class ZnW:
    #
    def __init__(self, N, coefs, W):
        self.N = N
        self.coefs = (np.polydiv(coefs, W)[1] % N).astype('int')
        self.W = W 
        
    #

    def __add__(self, other):
        if isinstance(other, (int, float)):
            result = self.coefs[:]
            result[-1] += other 
            return ZnW(self.N, result, self.W)
        
        maxLength = max(len(self.coefs), len(other.coefs))
        P = np.pad(self.coefs, (maxLength - len(self.coefs), 0))  
        Q = np.pad(other.coefs, (maxLength - len(other.coefs), 0))

        result = P + Q
        return ZnW(self.N, result, self.W)
    
    def __sub__(self, other):
        if isinstance(other, (int, float)):
            result = self.coefs[:]
            result[-1] -= other 
            return ZnW(self.N, result, self.W)
        
        maxLength = max(len(self.coefs), len(other.coefs))
        P = np.pad(self.coefs, (maxLength - len(self.coefs), 0))  
        Q = np.pad(other.coefs, (maxLength - len(other.coefs), 0))

        result = P - Q
        return ZnW(self.N, result, self.W)

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            result = np.polymul(self.coefs, [other])
        else:
            result = np.polymul(self.coefs, other.coefs) 
            return ZnW(self.N, result, self.W)
    
    def __rmul__(self, other):
        if isinstance(other, (int, float)):
            result = self.coefs * other
        else:
            result = np.polymul(self.coefs, other.coefs) 
        return ZnW(self.N, result, self.W)
           
    def __repr__(self):
        return f"{self.coefs}"
    
#end ZnW() class 

## Baby Kyber

Zaimplementuj poniższe elementy kryptosystemu Baby Kyber tak, aby osiągnąć jak największą skuteczność w testach (przy niezerowych błędach). Wymagana minimalna skuteczność to 60%.

Na razie zaimplementowałem dla przykładowych danych. Na dole pliku dla dowolnych.

In [110]:
W = [1, 0, 0, 0, 1]

A = np.array([
    [ZnW(17, [6, 16, 16, 11], W), ZnW(17, [9, 4, 6, 3], W)],
    [ZnW(17, [5, 3, 10, 1], W), ZnW(17, [6, 1, 9, 15], W)]
])

s = np.array([ZnW(17, [-1, -1, 1, 0], W), ZnW(17, [-1, 0, -1, 0], W)])
print(s)
e = np.array([ZnW(17, [1, 0, 0], W), ZnW(17, [1, -1, 0], W)])
t = np.dot(A, s.T) + e.T
print(t)

[[16 16  1  0] [16  0 16  0]]
[[16 15  0  7] [10 12 11  6]]


In [111]:
m = ZnW(17, [1, 0, 1, 1], W)
r = np.array([ZnW(17, [-1, 1, 0, 0], W), ZnW(17, [1, 1, 0, -1], W)])
e1 = np.array([ZnW(17, [1, 1, 0], W), ZnW(17, [1, 0, 0], W)])
e2 = np.array([ZnW(17, [-1, -1, 0, 0], W)])
u = np.dot(A.T, r) + e1 

In [112]:
print(np.dot(t.T, r).coefs)

[0 7 0 7]


In [113]:
print(np.dot(t.T, r) + e2[0])

[16  6  0  7]


In [114]:
v = np.dot(t.T, r) + e2[0] + 9 * m
v

[ 8  6  9 16]

In [115]:
m_n = v - np.dot(s.T, u)
m_n

[ 8 14  8  6]

In [116]:
def encrypt(coefs):
    m = ZnW(17, coefs, W)
    v = np.dot(t.T, r) + e2[0] + 9 * m
    return v - np.dot(s.T, u)

In [117]:
def roundToNearest(value):
    distTo0 = abs(value - 0)
    distTo9 = abs(value - 9)
    distTo17 = abs(value - 17)
    
    if distTo9 < distTo0 and distTo9 < distTo17:
        return 1
    else:
        return 0

In [118]:
def decrypt(znw):
    return list(map(lambda value : roundToNearest(value), znw.coefs))

In [119]:
import secrets as sc

success = 0
for i in range(1000):
    output = []
    
    m=[sc.choice((0,1)) for k in range(4)]

    c = encrypt(m)
    m_n = decrypt(c)
    if m_n == m:
        success += 1

print(f'Success rate: {success * 100 /1000} %')


Success rate: 100.0 %


### Generowanie klucza

Zaimplementuj funkcję `key_gen()` realizującą generowanie klucza w kryptosystemie Baby Kyber. Funkcja ma zwracać `A,t,s`. Przetestuj, czy dla podanych $A,s,e$ otrzymasz poprawny wielomian $t$.

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

$\mathbf{e}=(x^2,x^2-x)$

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

In [120]:
q = 23
k = 2 
n = 4 
W = [1, 0, 0, 0, 1]

In [121]:
def key_gen():
    # returns A, t, s
    A = np.array([[ZnW(q, np.random.randint(0, q, n), W) for _ in range(k)] for _ in range(k)])
    s = np.array([ZnW(q, np.random.choice([-1, 0, 1], size=n, p=[0.1, 0.8, 0.1]), W) for _ in range(k)])
    
    e = np.array([ZnW(q, np.random.choice([-1, 0, 1], size=n, p=[0.1, 0.8, 0.1]), W) for _ in range(k)])
    t = np.dot(A, s.T) + e.T
    return A, t, s

In [122]:
key_gen()

(array([[[ 2 13  9  0], [ 5  5  3 10]],
        [[13 14  3 10], [ 2  0 17  5]]], dtype=object),
 array([[0], [0]], dtype=object),
 array([[0], [0]], dtype=object))

### Szyfrowanie

Zaimplementuj funkcję `encrypt(A,t,m)` realizującą szyfrowanie w kryptosystemie Baby Kyber a gdzie wejściowe `m` jest w postaci listy. Funkcja ma zwracać szyfrogram `c`. Przetestuj poprawność działania na poniższych danych. 

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

$\mathbf{e_1}=(x^2+x,x^2)$

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11),8x^3+6x^2+9x+16)$

In [123]:
import math 

In [124]:
def encrypt(A, t, m):
    m = ZnW(17, m, W)
    r = np.array([ZnW(q, np.random.choice([-1, 0, 1], size=n, p=[0.1, 0.8, 0.1]), W) for _ in range(k)])
    e1 = np.array([ZnW(q, np.random.choice([-1, 0, 1], size=n, p=[0.1, 0.8, 0.1]), W) for _ in range(k)])
    e2 = np.array([ZnW(q, np.random.choice([-1, 0, 1], size=1, p=[0.1, 0.8, 0.1]), W)])

    u = np.dot(A.T, r) + e1 
    v = np.dot(t.T, r) + e2[0] + math.ceil(q / 2) * m
    
    c = (u, v)
    return c
# 

### Deszyfrowanie

Zaimplementuj funkcję `decrypt(c,s)` realizującą deszyfrowanie w kryptosystemie Baby Kyber. Funkcja ma zwracać ostateczną odszyfrowaną wiadomość `m_n`. Przetestuj działanie na poniższych danych.

$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$

$m_n=1\cdot x^3+0\cdot x^2+1\cdot x+1$


In [125]:
def roundToNearest(m_n, q):
    midpoint = math.ceil(q / 2)
    distTo0 = abs(m_n - 0)
    distToMid = abs(m_n - midpoint)
    distToQ = abs(m_n - q)
    
    if distToMid< distTo0 and distToMid < distToQ:
        return 1
    else:
        return 0
#

def decrypt(c, s):
    u, v = c 
    m_n = v - np.dot(s.T, u)

    return list(map(lambda value : roundToNearest(value, q), m_n.coefs))

### Testy

In [126]:
import secrets as sc

success = 0
for i in range(1000):
    output = []
    A,t,s = key_gen()
    
    m = [sc.choice((0,1)) for k in range(4)]
    
    c = encrypt(A,t,m)
    m_n = decrypt(c,s)

    if m_n == m:
        success += 1

print(f'Success rate: {success * 100 /1000} %')


Success rate: 75.4 %
